## 數據集來源：https://ade20k.csail.mit.edu/
## 檔案還蠻大，下載回來自己解壓 :D
http://sceneparsing.csail.mit.edu/data/ChallengeData2017/images.tar  850MB
http://sceneparsing.csail.mit.edu/data/ChallengeData2017/annotations_instance.tar 86.2MB

#### 下面格子的這些東西要直接下載
#### https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

In [10]:
!chcp 65001
!echo 中文在我的電腦顯示會是亂碼耶  

Active code page: 65001


The value specified in an AutoRun registry key could not be parsed.


中文在我的電腦顯示會是亂碼耶


The value specified in an AutoRun registry key could not be parsed.


### 下載windows的wget
https://eternallybored.org/misc/wget

放到 C:\Windows\System32

In [11]:
#!wget --quiet https://raw.githubusercontent.com/pytorch/vision/release/0.12/references/detection/engine.py
#!wget --quiet https://raw.githubusercontent.com/pytorch/vision/release/0.12/references/detection/utils.py
#!wget --quiet https://raw.githubusercontent.com/pytorch/vision/release/0.12/references/detection/transforms.py
#!wget --quiet https://raw.githubusercontent.com/pytorch/vision/release/0.12/references/detection/coco_eval.py
#!wget --quiet https://raw.githubusercontent.com/pytorch/vision/release/0.12/references/detection/coco_utils.py
#### 需要安裝的coco資料集API
# %pip install pycocotools

In [12]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from engine import train_one_epoch, evaluate
import utils
import transforms as T
from pycocotools.coco import COCO
import glob, os, configparser, json
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [13]:
# Load configuration
config = configparser.ConfigParser()
config.read('config.ini')
# Load parameters from config file
#root = '/Users/leonjye/Documents/MachineLearingData'
root = config.get('DEFAULT', 'root_dir')
IMAGE_ROOT = os.path.join(root, 'ChallengeData2017')

In [14]:
def Glob(x, extns=None):
    files = glob.glob(x + "/*") if "*" not in x else glob.glob(x)
    if extns:
        if isinstance(extns, str):
            extns = extns.split(",")
        files = [f for f in files if any([f.endswith(ext) for ext in extns])]
    return files

In [15]:
all_images = Glob(os.path.join(IMAGE_ROOT, 'images/training'))
all_annots = Glob(os.path.join(IMAGE_ROOT, 'annotations_instance/training'))
all_images

['C:\\Users\\leonj\\Documents\\Data\\ChallengeData2017\\images/training\\ADE_train_00000001.jpg',
 'C:\\Users\\leonj\\Documents\\Data\\ChallengeData2017\\images/training\\ADE_train_00000002.jpg',
 'C:\\Users\\leonj\\Documents\\Data\\ChallengeData2017\\images/training\\ADE_train_00000003.jpg',
 'C:\\Users\\leonj\\Documents\\Data\\ChallengeData2017\\images/training\\ADE_train_00000004.jpg',
 'C:\\Users\\leonj\\Documents\\Data\\ChallengeData2017\\images/training\\ADE_train_00000005.jpg',
 'C:\\Users\\leonj\\Documents\\Data\\ChallengeData2017\\images/training\\ADE_train_00000006.jpg',
 'C:\\Users\\leonj\\Documents\\Data\\ChallengeData2017\\images/training\\ADE_train_00000007.jpg',
 'C:\\Users\\leonj\\Documents\\Data\\ChallengeData2017\\images/training\\ADE_train_00000008.jpg',
 'C:\\Users\\leonj\\Documents\\Data\\ChallengeData2017\\images/training\\ADE_train_00000009.jpg',
 'C:\\Users\\leonj\\Documents\\Data\\ChallengeData2017\\images/training\\ADE_train_00000010.jpg',
 'C:\\Users\\leonj\\

In [ ]:
f = 'ADE_train_00014301'

im = read(find(f, all_images), 1)
an = read(find(f, all_annots), 1).transpose(2,0,1)
r,g,b = an
nzs = np.nonzero(r==4) # 4 stands for person
instances = np.unique(g[nzs])
masks = np.zeros((len(instances), *r.shape))
for ix,_id in enumerate(instances):
    masks[ix] = g==_id

subplots([im, *masks], sz=20)